# From Scratch to Production: Oceananigans.jl

In the previous notebooks we:
1. **GPU Computing** — learned how to write portable GPU kernels with KernelAbstractions.jl
2. **Navier-Stokes from Scratch** — built a 2D incompressible solver by hand: staggered C-grid, finite differences, FFT pressure solver, projection method, multiple dispatch for advection schemes

Now we'll see how **Oceananigans.jl** — a production ocean modeling package — solves the same equations and much more. Under the hood, Oceananigans uses the exact same ingredients:
- **Arakawa C-grid** (staggered grid) → `RectilinearGrid`
- **Projection method** with pressure Poisson solver → `NonhydrostaticModel`
- **KernelAbstractions.jl** for CPU/GPU portability → just pass `GPU()` instead of `CPU()`
- **Multiple dispatch** for extensible physics → advection schemes, closures, buoyancy models

But it adds capabilities we didn't build: 3D, boundary conditions, tracer transport, buoyancy, adaptive time stepping, output management, and more.

## Cabbeling: When Mixing Creates Density

In this notebook we simulate **cabbeling** — convection driven by the nonlinearity of the equation of state of water.
We start with a stable fluid: hot water (7.55 °C) above cold water (1 °C), both at the **same density**.
The fluid is initially at rest, but when mixing occurs at the interface, the resulting intermediate temperatures
correspond to *higher* density (fresh water is densest near 4 °C). The mixed water sinks, driving convection from an initially stable state.

In [1]:
using Pkg
Pkg.activate("./")

using Oceananigans
using Oceananigans.Models: seawater_density
using SeawaterPolynomials: TEOS10EquationOfState
using CairoMakie

  Activating project at `~/development/JuliaLessons`


## Step 1: Choose the Architecture

Remember how our hand-built solver used `get_backend(A)` to detect CPU vs GPU?
Oceananigans makes this a first-class concept. To run on a GPU, just replace `CPU()` with `GPU()`.
Everything — grids, fields, kernels — automatically adapts.

In [2]:
arch = CPU()

CPU()

## Step 2: Define the Grid

In our hand-built solver, we created `Grid(Nx, Ny, Lx, Ly)` and manually allocated staggered arrays.
Oceananigans' `RectilinearGrid` does all of this automatically — it sets up the Arakawa C-grid with
the correct staggered locations for `u`, `v`, `w`, and cell-center fields.

The key addition is **topology**: each direction can be `Periodic`, `Bounded`, or `Flat`.
Our solver only supported `Periodic`. Here we use `Bounded` walls in x and z (a vertical slice),
with `Flat` in y (making it 2D in the x-z plane).

In [3]:
Nx, Ny = 300, 300
grid = RectilinearGrid(arch,
                       size = (Nx, Ny),
                       x = (0, 0.5),
                       z = (-0.5, 0.0),
                       topology = (Bounded, Flat, Bounded))

300×1×300 RectilinearGrid{Float64, Bounded, Flat, Bounded} on CPU with 3×0×3 halo
├── Bounded  x ∈ [0.0, 0.5]  regularly spaced with Δx=0.00166667
├── Flat y                   
└── Bounded  z ∈ [-0.5, 0.0] regularly spaced with Δz=0.00166667

## Step 3: Buoyancy — New Physics Beyond Our Solver

Our hand-built solver only had velocity and pressure. Oceananigans can couple the momentum equation
to **buoyancy** through an equation of state.

Here we use the **TEOS-10** equation of state — the international standard for seawater thermodynamics.
This is a nonlinear function $\rho(T, S, p)$ that relates density to temperature, salinity, and pressure.
This nonlinearity is exactly what drives cabbeling: even though $T_1 = 1°$C and $T_2 = 7.55°$C have the
same density, their average $T \approx 4°$C is **denser** (fresh water has maximum density near 4°C).

In [4]:
equation_of_state = TEOS10EquationOfState(reference_density=1000)
buoyancy = SeawaterBuoyancy(; equation_of_state,
                              constant_salinity=0, 
                              gravitational_acceleration=9.80655)

SeawaterBuoyancy{Float64}:
├── gravitational_acceleration: 9.80655
├── constant_salinity: 0.0
└── equation_of_state: BoussinesqEquationOfState{Float64}

## Step 4: Build the Model

This is the equivalent of our `NavierStokesModel`. Compare:

| Our solver | Oceananigans |
|------------|-------------|
| `NavierStokesModel(grid, ν; advection=Centered())` | `NonhydrostaticModel(grid; advection=WENO(), ...)` |
| `FFTPoissonSolver` for pressure | Built-in pressure solver (auto-selected) |
| `Centered()`, `Upwind()` dispatch | `CenteredSecondOrder()`, `WENO(order=7)`, ... |
| Forward Euler | `RungeKutta3` (default) |
| Explicit `ν` parameter | `closure = ScalarDiffusivity(ν=...)` |

Here we use **WENO(order=7)** — a high-order Weighted Essentially Non-Oscillatory scheme.
Remember how we used multiple dispatch to swap between `Centered` and `Upwind`?
Oceananigans does the same: WENO is just another advection type that dispatches to different reconstruction methods.

With high-order WENO providing implicit numerical dissipation, we don't need an explicit viscosity closure.

In [5]:
model = NonhydrostaticModel(grid;
                            buoyancy,
                            advection=WENO(order=7),
                            tracers=:T)

┌ Warning: Inflating model grid halo size to (4, 0, 4) and recreating grid. Note that an ImmersedBoundaryGrid requires an extra halo point in all non-flat directions compared to a non-immersed boundary grid.
└ @ Oceananigans.Models.NonhydrostaticModels ~/.julia/packages/Oceananigans/t5Rdl/src/Models/NonhydrostaticModels/nonhydrostatic_model.jl:311


NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 300×1×300 RectilinearGrid{Float64, Bounded, Flat, Bounded} on CPU with 4×0×4 halo
├── timestepper: RungeKutta3TimeStepper
├── advection scheme: WENO{4, Float64, Float32}(order=7)
├── tracers: T
├── closure: Nothing
├── buoyancy: SeawaterBuoyancy with g=9.80655 and BoussinesqEquationOfState{Float64} with ĝ = NegativeZDirection()
└── coriolis: Nothing

## Step 5: Initial Conditions

In our hand-built solver, we initialized velocity fields directly by writing into the arrays.
Oceananigans provides a convenient `set!` function that accepts either arrays or functions of coordinates.

We initialize:
- **Temperature**: hot water ($T_2 = 7.55°$C) and cold water ($T_1 = 1°$C) arranged in a pattern
- **Velocity**: tiny random perturbations to trigger the instability

For fun, we'll initialize the temperature pattern from a logo image!

In [52]:
T₁, T₂ = 1, 7.55 # ᵒC
Ξᵢ = (x, z) -> 1e-4 * randn()

using FileIO

download("https://www.ogs.it/themes/custom/italiagov/assets/xlogo_en.png.pagespeed.ic.epDevB0Np8.webp", "logo-ogs.png")

img   = FileIO.load("logo-ogs.png")
alpha = getproperty.(img, :alpha) .|> Float64
alpha = reverse(alpha', dims=2)
alpha = alpha[451:4:1650, 1:2:end]
Tᵢ = [ifelse(alpha[i, j] == 0, T₁, T₂) for i in 1:Nx, j in 1:Ny]
Tᵢ = Oceananigans.on_architecture(arch, Tᵢ)

set!(model, T=Tᵢ, u=Ξᵢ, w=Ξᵢ)

┌ Warning: Reshaped 300×300 Matrix{Real} to set! its data to 
│ 300×1×300 Field{Center, Center, Center} on RectilinearGrid on CPU
└ @ Oceananigans.Fields ~/.julia/packages/Oceananigans/LrSXP/src/Fields/set!.jl:113


## Step 6: Simulation

In our solver, we called `run!(model, Δt, Tfinal)` in a simple loop.
Oceananigans wraps this in a `Simulation` object that manages time stepping, callbacks, and output.

In [53]:
simulation = Simulation(model; Δt=0.1, stop_time=100)

Simulation of NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 100.000 ms
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: 1.667 minutes
├── Stop iteration: Inf
├── Wall time limit: Inf
├── Minimum relative step: 0.0
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => Callback of stop_time_exceeded on IterationInterval(1)
│   ├── stop_iteration_exceeded => Callback of stop_iteration_exceeded on IterationInterval(1)
│   ├── wall_time_limit_exceeded => Callback of wall_time_limit_exceeded on IterationInterval(1)
│   └── nan_checker => Callback of NaNChecker for u on IterationInterval(100)
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

### Progress callback

Oceananigans supports **callbacks** — functions that run during the simulation.
This is much more flexible than hardcoding print statements in a time loop.

In [54]:
wall_time = [time_ns()]

function progress(sim)
    u, v, w = sim.model.velocities
    step_time = (time_ns() - wall_time[1]) * 1e-9
    @info string("Iter: ", iteration(sim), ", wall_time: ", prettytime(step_time), " time: ", prettytime(sim), ", max(w): ", maximum(abs, w))
    wall_time[1] = time_ns()
end

add_callback!(simulation, progress, IterationInterval(100))

### Adaptive time stepping

Our solver used a fixed $\Delta t$. Oceananigans can automatically adjust the time step
to satisfy a CFL condition — crucial for flows where velocity magnitudes change over time.

In [55]:
conjure_time_step_wizard!(simulation, cfl=0.7)

### Output writers

In our solver we collected snapshots in a Julia array.
Oceananigans provides `OutputWriter`s that save fields to disk (JLD2, NetCDF) on a schedule.
This is essential for large simulations where you can't keep everything in memory.

Note how `seawater_density(model)` computes $\rho$ from $T$ via the equation of state — this is a
**computed field** (a lazy operation that evaluates on demand).

In [56]:
ρ = seawater_density(model)
T = model.tracers.T

output_writer = JLD2OutputWriter(model, (; ρ, T),
                                 filename = "cabbeling",
                                 schedule = TimeInterval(1),
                                 overwrite_existing = true)

simulation.output_writers[:jld2] = output_writer

JLD2OutputWriter scheduled on TimeInterval(1 second):
├── filepath: cabbeling.jld2
├── 2 outputs: (ρ, T)
├── array type: Array{Float32}
├── including: [:grid, :coriolis, :buoyancy, :closure]
├── file_splitting: NoFileSplitting
└── file size: 2.2 MiB

## Step 7: Run!

Just like `run!(model, Δt, Tfinal)` in our solver, but with all the bells and whistles:
adaptive time stepping, callbacks, output writing — all handled automatically.

In [57]:
run!(simulation)

[ Info: Initializing simulation...
[ Info: Iter: 0, wall_time: 5.359 seconds time: 0 seconds, max(w): 0.00031973806
[ Info:     ... simulation initialization complete (86.018 ms)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (105.578 ms).
[ Info: Iter: 100, wall_time: 7.775 seconds time: 15.172 seconds, max(w): 0.0063787918
[ Info: Iter: 200, wall_time: 7.707 seconds time: 26.774 seconds, max(w): 0.009304275
[ Info: Iter: 300, wall_time: 7.501 seconds time: 34.545 seconds, max(w): 0.010106918
[ Info: Iter: 400, wall_time: 8.791 seconds time: 42.000 seconds, max(w): 0.011301949
[ Info: Iter: 500, wall_time: 8.259 seconds time: 49.071 seconds, max(w): 0.015265129
[ Info: Iter: 600, wall_time: 9.410 seconds time: 55.778 seconds, max(w): 0.014972569
[ Info: Iter: 700, wall_time: 7.842 seconds time: 1.027 minutes, max(w): 0.017682273
[ Info: Iter: 800, wall_time: 7.312 seconds time: 1.117 minutes, max(w): 0.021433322
[ Info: Iter: 900, wall_time: 7.343 se

## Visualizing the Results

Oceananigans saves output as `FieldTimeSeries` objects that can be loaded from disk.
We use CairoMakie to create an animation of temperature and density evolution.

In [23]:
using CairoMakie

ρt = FieldTimeSeries("cabbeling.jld2", "ρ")
Tt = FieldTimeSeries("cabbeling.jld2", "T")

Nt = length(ρt)
Nx = size(ρt, 1)

i = Int(Nx / 2)
n = Observable(length(ρt.times))
ρ = @lift interior(ρt[$n], :, 1, :)
T = @lift interior(Tt[$n], :, 1, :)
x, y, z = nodes(ρt)

set_theme!(Theme(fontsize=12))
fig = Figure(size=(1000, 400))

ρrange = (minimum(ρt[1]), maximum(ρt))

axT = Axis(fig[1, 2], xlabel="x (m)", ylabel="z (m)")
xlims!(axT,  0, 0.5)
ylims!(axT, -0.5, 0)

axρ = Axis(fig[1, 3], xlabel="x (m)", ylabel="z (m)")
xlims!(axρ,  0, 0.5)
ylims!(axρ, -0.5, 0)

hm = heatmap!(axT, x, z, T, colormap=:magma, colorrange=(1.55, 7))
Colorbar(fig[1, 1], hm, label="Temperature (ᵒC)", flipaxis=false)

hm = heatmap!(axρ, x, z, ρ, colormap=Makie.Reverse(:grays), colorrange=ρrange)
Colorbar(fig[1, 4], hm, label="Density (kg m⁻³)")

record(fig, "cabbeling_2d.mp4", 1:Nt, framerate=5) do nn
    mod(nn, 10) == 0 && @info "Drawing frame $nn of $Nt..."
    n[] = nn
end

[ Info: Drawing frame 10 of 101...
[ Info: Drawing frame 20 of 101...
[ Info: Drawing frame 30 of 101...
[ Info: Drawing frame 40 of 101...
[ Info: Drawing frame 50 of 101...
[ Info: Drawing frame 60 of 101...
[ Info: Drawing frame 70 of 101...
[ Info: Drawing frame 80 of 101...
[ Info: Drawing frame 90 of 101...
[ Info: Drawing frame 100 of 101...


"cabbeling_2d.mp4"

## What We've Seen

Starting from raw GPU kernels and building a Navier-Stokes solver by hand,
we've now seen how Oceananigans wraps the same core ideas into a production tool:

| Concept | Our Solver | Oceananigans |
|---------|-----------|--------------|
| Grid | `Grid(Nx, Ny, Lx, Ly)` | `RectilinearGrid(arch; size, x, y, topology)` |
| Staggering | Manual `u[Nx, Ny]`, `v[Nx, Ny]` | Automatic C-grid with halo regions |
| Pressure | `FFTPoissonSolver` + `solve!` | Built-in (auto-selected for topology) |
| Advection | `Centered()`, `Upwind()` via dispatch | `WENO()`, `CenteredSecondOrder()`, ... |
| Time stepping | Forward Euler | `RungeKutta3` (default), `QuasiAdamsBashforth2` |
| Viscosity | Explicit `ν` parameter | `closure = ScalarDiffusivity(ν=...)` or LES closures |
| GPU support | `CuArray` + KernelAbstractions | Just pass `GPU()` as architecture |
| **New: Buoyancy** | — | Equation of state coupling |
| **New: Tracers** | — | `tracers = :T` (or `:S`, or any name) |
| **New: Output** | In-memory arrays | `JLD2OutputWriter`, `NetCDFOutputWriter` |
| **New: Adaptive Δt** | — | `conjure_time_step_wizard!(sim, cfl=0.7)` |

The key takeaway: **understanding the fundamentals** (staggered grids, projection method, GPU kernels)
gives you the foundation to use — and contribute to — production-grade tools like Oceananigans.

## Exercise: Ocean Salinity

In this simulation we used `constant_salinity = 0` (fresh water).
Real ocean water has salinity around 35 g/kg.

**Question:** What do you think will happen if we increase the salinity to match ocean conditions?
Will cabbeling be stronger or weaker? Will it still occur at all?

**Your prediction:** *(think before you code!)*

> ...

### Step 1: Explore the equation of state

Use `SeawaterPolynomials` to compute the density at different temperatures for both
fresh water (S = 0) and seawater (S = 35). Plot $\rho(T)$ for both cases.

**Hint:** the temperature of maximum density for fresh water is ~4 °C.
What happens to this maximum at S = 35?

In [ ]:
# Step 1: Plot ρ(T) for S=0 and S=35
# Hint: you can use Oceananigans.Models.seawater_density or 
# SeawaterPolynomials.TEOS10.ρ to compute density


### Step 2: Run the simulation with ocean salinity

Modify the simulation above to use `constant_salinity = 35`.
You'll need to find two temperatures $T_1$ and $T_2$ that have **equal density** at $S = 35$.

**Hints:**
- The density maximum disappears at high salinity — seawater density increases monotonically as temperature decreases
- Try choosing $T_1$ and $T_2$ symmetric around some temperature and check if $\rho(T_1, S=35) \approx \rho(T_2, S=35)$
- Does cabbeling still occur? Is it stronger or weaker than in fresh water?

In [ ]:
# Step 2: Re-run the cabbeling simulation with constant_salinity = 35
# Copy and modify the setup from above — you only need to change:
#   1. The buoyancy (constant_salinity = 35)
#   2. The initial temperatures T₁ and T₂ (find equal-density pair at S = 35)
